### Использование моделей

In [1]:
# импорт библиотек

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import catboost
from catboost import CatBoostClassifier
from catboost.utils import eval_metric

### Загрузка тестовых данных

In [271]:
# загрузка данных y_train

test = pq.ParquetFile(r'C:\Hackathon\54\pred\x_test_pred.parquet')
test = test.read_row_group(0)

df = []
for col in test.column_names:
    if col == 'DT':
        df.append(test[col].to_pandas()) 
    else:
        df.append(test[col].to_pandas().astype('float32'))

X_test_pool= pd.concat(df, axis=1)

del test
del df
del col

In [273]:
X_test_pool = X_test_pool.set_index('DT')

In [348]:
# Загрузка тестовых интервалов

test_intervals = pd.read_excel(r'C:\Hackathon\54\test\test_intervals.xlsx')

In [349]:
test_intervals = test_intervals.drop(test_intervals.columns[0], axis = 1)

In [350]:
test_intervals

,start,finish,machine,tm
0,2022-01-01 10:54:53,2022-01-01 16:43:39,NaN,NaN
1,2022-01-02 12:26:42,2022-01-02 15:16:06,NaN,NaN
2,2022-01-04 12:06:07,2022-01-04 15:04:34,NaN,NaN
3,2022-01-04 20:13:57,2022-01-05 00:19:40,NaN,NaN
4,2022-01-07 09:05:16,2022-01-07 14:05:15,NaN,NaN
...,...,...,...,...
184,2023-04-07 15:36:42,2023-04-07 19:27:38,NaN,NaN
185,2023-04-09 19:55:43,2023-04-09 23:12:53,NaN,NaN
186,2023-04-10 18:08:25,2023-04-10 21:47:02,NaN,NaN
187,2023-04-18 02:28:43,2023-04-18 06:38:47,NaN,NaN


### Загрузка модели определения дефекта М1

In [113]:
# Список техмест для которых определяем дефекты М1
defect_M1_list = ['Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9', 
               'Y_ЭКСГАУСТЕР А/М №9_ТР-Р ТМ-6300-10/6 ЭКСГ. №9', 
               'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5', 
               'Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5', 
               'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №5', 
               'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №1', 
               'Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1', 
               'Y_ЭКСГАУСТЕР А/М №5_ЭКСГАУСТЕР А/М №5', 
               'Y_ЭКСГАУСТЕР А/М №5_РОТОР ЭКСГ. №5', 
               'Y_ЭКСГАУСТЕР А/М №5_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №5', 
               'Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ2', 
               'Y_ЭКСГАУСТЕР А/М №8_РОТОР ЭКСГ. №8', 
               'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8', 
               'Y_ЭКСГАУСТЕР А/М №8_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ СПВД-М10-400-5', 
               'Y_ЭКСГАУСТЕР А/М №8_ЭКСГАУСТЕР А/М №8', 
               'Y_ЭКСГАУСТЕР А/М №8_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ8 ВУ1', 
               'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №8', 
               'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4', 
               'Y_ЭКСГАУСТЕР А/М №4_ЭКСГАУСТЕР А/М №4', 
               'Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №7', 
               'Y_ЭКСГАУСТЕР А/М №7_КОРПУС ЭКСГ. №7', 
               'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ №2', 
               'Y_ЭКСГАУСТЕР А/М №7_РОТОР ЭКСГ. №7', 
               'Y_ЭКСГАУСТЕР А/М №7_ЭКСГАУСТЕР А/М №7', 
               'Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №7', 
               'Y_ЭКСГАУСТЕР А/М №6_ЗАДВИЖКА ЭКСГ. №6', 
               'Y_ЭКСГАУСТЕР А/М №6_ГАЗОВАЯ ЗАДВИЖКА ЭКСГАУСТЕРА А/М №6', 
               'Y_ЭКСГАУСТЕР А/М №6_ЭКСГАУСТЕР А/М №6']

In [276]:
model_M1 = CatBoostClassifier()
model_M1.load_model('model_detect_defect_m1_1_best.bin')

In [115]:
model_M1.get_params()

{'use_best_model': True,
 'random_strength': 1,
 'od_wait': 20,
 'leaf_estimation_method': 'Gradient',
 'verbose': 1,
 'iterations': 1600,
 'bagging_temperature': 1,
 'save_snapshot': True,
 'custom_metric': ['F1'],
 'loss_function': 'MultiLogloss',
 'l2_leaf_reg': 4,
 'depth': 10,
 'learning_rate': 0.06,
 'random_seed': 42}

In [277]:
y_pred_M1 = model_M1.predict(X_test_pool)

In [279]:
y_pred_M1_df = pd.DataFrame(y_pred_M1, columns = defect_M1_list)

In [281]:
y_pred_M1_df = y_pred_M1_df.set_index(X_test_pool.index)

In [340]:
for i in range(len(test_intervals)):
    df = y_pred_M1_df.loc[test_intervals['start'][i]:test_intervals['finish'][i]]
    if (df == 1).sum().sum() > 0:
        print('1')
    for i in range(len(df.columns)):
        gr = df.groupby(by=df.columns[i])
        if 1 in gr:
            print(gr.get_group(1).index[0], df.columns[i])



In [351]:
test_intervals.iloc[::,2] = 0
test_intervals.iloc[::,3] = 0

C:\Users\l-pla\AppData\Local\Temp\ipykernel_14268\1259938610.py:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test_intervals.iloc[::,2] = 0
C:\Users\l-pla\AppData\Local\Temp\ipykernel_14268\1259938610.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test_intervals.iloc[::,3] = 0


In [354]:
test_intervals.to_excel('test_intervals_pred.xlsx')

### Загрузка модели определения дефекта М3

In [356]:
# Список техмест для которых определяем дефекты М1
defect_M3_list = ['Y_ЭКСГАУСТЕР А/М №9_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_МАСЛОСТАНЦИЯ ЖИДКОЙ СМАЗКИ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_КОРПУС ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_РОТОР ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ТР-Р ТМ-6300-10/6 ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ №2', 
                  'Y_ЭКСГАУСТЕР А/М №9_ГСМ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_КОЖУХ МУФТЫ ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №9_ПОДШИПНИК ОПОРНЫЙ №1', 
                  'Y_ЭКСГАУСТЕР А/М №9_ЗАДВИЖКА ЭКСГ. №9', 
                  'Y_ЭКСГАУСТЕР А/М №5_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_КОРПУС ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ЗАДВИЖКА ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №2', 
                  'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (РЕЗ.) ЭКСГ. №5',
                  'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.1', 
                  'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.3',
                  'Y_ЭКСГАУСТЕР А/М №5_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГАУСТЕРА №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№5 Т.2',
                  'Y_ЭКСГАУСТЕР А/М №5_ПОДШИПНИК ОПОРНЫЙ №1', 
                  'Y_ЭКСГАУСТЕР А/М №5_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №5',
                  'Y_ЭКСГАУСТЕР А/М №5_ТИРИСТ. ВОЗБУДИТЕЛЬ ВТ-РЭМ-400 ЭКСГ5 ВУ1',
                  'Y_ЭКСГАУСТЕР А/М №5_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ГСМ ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ТР-Р ТМ-4000-10/6 ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_КЛ1 ТР№5 ДО ЭД ЭКСГАУСТЕРА №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_КЛ2 ТР№5 ДО ЭД ЭКСГАУСТЕРА №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №5',
                  'Y_ЭКСГАУСТЕР А/М №5_РОТОР ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_ДВИГАТЕЛЬ ПУСКОВОГО МАСЛОНАСОСА ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №5_КОЖУХ МУФТЫ ЭКСГ. №5', 
                  'Y_ЭКСГАУСТЕР А/М №8_РОТОР ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №8',
                  'Y_ЭКСГАУСТЕР А/М №8_КОРПУС ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №8',
                  'Y_ЭКСГАУСТЕР А/М №8_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№8 Т.3', 
                  'Y_ЭКСГАУСТЕР А/М №8_ВК 310С ВИБРОПРЕОБРАЗОВАТЕЛЬ ЭКСГ.№8 Т.4',
                  'Y_ЭКСГАУСТЕР А/М №8_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №8',
                  'Y_ЭКСГАУСТЕР А/М №8_КЛ2 ТР№8 ДО ЭД ЭКСГАУСТЕРА №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_КЛ1 ТР№8 ДО ЭД ЭКСГАУСТЕРА №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_КОЖУХ МУФТЫ ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ГСМ ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ №2', 
                  'Y_ЭКСГАУСТЕР А/М №8_ЗАПОРНАЯ АРМАТУРА ЭКСГАУСТЕРА №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ЗАДВИЖКА ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №8',
                  'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (РЕЗ.) ЭКСГ. №8', 
                  'Y_ЭКСГАУСТЕР А/М №8_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№8', 
                  'Y_ЭКСГАУСТЕР А/М №8_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №8',
                  'Y_ЭКСГАУСТЕР А/М №8_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №8', 
                  'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №4',
                  'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_УЛИТА ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №4',
                  'Y_ЭКСГАУСТЕР А/М №4_ЭЛЕКТРОАППАРАТУРА ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №2 ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ЗАДВИЖКА ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ГСМ ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_РОТОР ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №4',
                  'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №4',
                  'Y_ЭКСГАУСТЕР А/М №4_ЗАП. И РЕГ. АРМАТУРА ЭКСГ.№4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ПОДШИПНИК ОПОРНЫЙ №1 ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ЭЛ/ДВИГАТЕЛЬ ГАЗ. ЗАДВИЖКИ ЭКСГ. №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_КЛ1 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4',
                  'Y_ЭКСГАУСТЕР А/М №4_КЛ2 ТР№4 ДО ЭД ЭКСГАУСТЕРА №4', 
                  'Y_ЭКСГАУСТЕР А/М №4_ТР-Р ТМ-4000-10/6 ЭКСГ. №4',
                  'Y_ЭКСГАУСТЕР А/М №7_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_КОРПУС ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ №1',
                  'Y_ЭКСГАУСТЕР А/М №7_ЗАДВИЖКА ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ №2',
                  'Y_ЭКСГАУСТЕР А/М №7_РОТОР ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №7',
                  'Y_ЭКСГАУСТЕР А/М №7_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №7',
                  'Y_ЭКСГАУСТЕР А/М №7_ГСМ ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_КОЖУХ МУФТЫ ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_МАСЛОПРОВОДЫ ЭКСГАУСТЕРА №7',
                  'Y_ЭКСГАУСТЕР А/М №7_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №7',
                  'Y_ЭКСГАУСТЕР А/М №7_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №7', 
                  'Y_ЭКСГАУСТЕР А/М №7_МАСЛОНАСОС ШЕСТЕРЕНЧАТЫЙ (ПУСК.) ЭКСГ.№7', 
                  'Y_ЭКСГАУСТЕР А/М №7_МАСЛЯНЫЙ ФИЛЬТР ЭКСГАУСТЕРА №7',
                  'Y_ЭКСГАУСТЕР А/М №7_ТИРИСТОРНЫЙ ВОЗБУДИТЕЛЬ ТВ-400 ЭКСГ7 ВУ1', 
                  'Y_ЭКСГАУСТЕР А/М №6_ЭЛЕКТРОДВИГАТЕЛЬ ДСПУ-140-84-4 ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_КОРПУС ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_ЗАДВИЖКА ЭКСГ. №6',
                  'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ №2', 
                  'Y_ЭКСГАУСТЕР А/М №6_РЕДУКТОР ГАЗ. ЗАДВИЖКИ ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНО-УПОРНЫЙ ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_МАСЛОБАК ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ №1', 
                  'Y_ЭКСГАУСТЕР А/М №6_КОЖУХ МУФТЫ ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_РОТОР ЭКСГ. №6', 
                  'Y_ЭКСГАУСТЕР А/М №6_ЭЛЕКТРООБОРУДОВАНИЯ ЭКСГ №6',
                  'Y_ЭКСГАУСТЕР А/М №6_МАСЛООХЛАДИТЕЛЬ  М-05-1 ЭКСГ. №6',
                  'Y_ЭКСГАУСТЕР А/М №6_МАСЛОНАСОС РАБОЧИЙ ЭКСГ. №6',
                  'Y_ЭКСГАУСТЕР А/М №6_ПОДШИПНИК ОПОРНЫЙ ЭКСГ. №6']

In [355]:
model_M3 = CatBoostClassifier()
model_M3.load_model('model_detect_defect.bin')

In [357]:
model_M3.get_params()

{'use_best_model': True,
 'rsm': 0.5,
 'leaf_estimation_iterations': 5,
 'bootstrap_type': 'Bernoulli',
 'od_wait': 10,
 'snapshot_file': 'model_defect_2.bkp',
 'boosting_type': 'Plain',
 'iterations': 100,
 'verbose': 1,
 'max_ctr_complexity': 1,
 'snapshot_interval': 60,
 'save_snapshot': True,
 'custom_metric': ['F1'],
 'loss_function': 'MultiLogloss',
 'subsample': 0.5,
 'learning_rate': 0.1,
 'random_seed': 35}

In [358]:
y_pred_M3 = model_M3.predict(X_test_pool)

In [360]:
dfm3 = pd.DataFrame(y_pred_M3, index=X_test_pool.index, columns=defect_M3_list)
dfm3.shape

(4008961, 111)

In [436]:
dfm3 = dfm3.replace(1,2)


In [439]:
dfm3.to_parquet('dfm3.parquet')

In [372]:
list_M3 = []
for i in range(len(test_intervals)):
    df = dfm3.loc[test_intervals['start'][i]:test_intervals['finish'][i]]
    for i in range(len(df.columns)):
        gr = df.groupby(by=df.columns[i])
        if 1 in gr.groups:
            list_M3.append((gr.get_group(1).index[0], df.columns[i]))

In [374]:
df_M3 = pd.DataFrame(list_M3)
df_M3.shape

(8921, 2)

In [391]:
df_M3 = df_M3.set_index(df_M3.columns[0])

In [ ]:
df_M3 = df_M3.replace(1,2)

In [395]:
df_M3.groupby(df_M3.columns[0]).

{'Y_ЭКСГАУСТЕР А/М №4_ГСМ ЭКСГ. №4': [2022-01-01 10:55:00, 2022-01-02 12:26:50, 2022-01-04 12:06:10, 2022-01-04 20:14:00, 2022-01-07 09:05:20, 2022-01-08 08:06:30, 2022-01-09 03:32:20, 2022-01-10 23:27:50, 2022-01-16 20:01:10, 2022-01-19 02:24:50, 2022-01-19 15:03:20, 2022-01-24 05:57:30, 2022-01-24 23:57:00, 2022-01-29 23:33:50, 2022-02-06 18:05:30, 2022-02-12 16:37:10, 2022-02-24 14:52:40, 2022-02-25 03:47:00, 2022-03-03 08:18:50, 2022-03-06 22:22:10, 2022-03-11 06:06:40, 2022-03-15 04:28:00, 2022-03-26 15:40:00, 2022-03-28 08:13:40, 2022-05-01 11:47:50, 2022-05-30 09:48:30, 2022-05-30 15:48:20, 2022-06-22 09:24:50, 2022-07-09 07:56:40, 2022-07-22 13:18:40, 2022-07-26 01:52:30, 2022-07-27 04:41:00, 2022-08-05 11:48:50, 2022-08-15 23:37:30, 2022-09-05 16:11:10, 2022-10-30 08:19:50, 2022-10-31 06:31:50, 2022-10-31 12:46:40, 2022-11-02 15:35:40, 2022-11-05 02:45:50, 2022-11-05 08:16:30, 2022-11-22 03:58:00, 2022-11-22 03:58:00, 2022-11-24 01:32:40, 2022-11-26 12:35:50, 2022-11-26 21:21:

In [426]:
dm = y_pred_M1_df.max(axis=1)
gr = dm.groupby(by=dm.values)
time_to_defect = []
gg = gr.groups[1]
for a, b in zip(gg, gg[1::]):
    time_to_defect += [round((b-a).value*0.000000001)]

In [431]:
time_to_defect_df = pd.DataFrame(time_to_defect, columns = ['time to M1'])

In [433]:
time_to_defect_df.to_excel('time_to_defect_df.xlsx')